In [1]:



import numpy as np
import math

def get_material_properties():
    # Get material properties from user
    E1 = float(input("Enter E1 for lamina (GPa): "))
    E2 = float(input("Enter E2 for lamina (GPa): "))
    G12 = float(input("Enter G12 for lamina (GPa): "))
    nu12 = float(input("Enter nu12 for lamina: "))
    alpha_1 = float(input("Enter coefficient of thermal expansion alpha_1 (1/°C): "))
    alpha_2 = float(input("Enter coefficient of thermal expansion alpha_2 (1/°C): "))
    alpha_12 = float(input("Enter coefficient of thermal expansion alpha_12 (1/°C): "))
    beta_1 = float(input("Enter moisture expansion coefficient beta_1 (%/°C): "))
    beta_2 = float(input("Enter moisture expansion coefficient beta_2 (%/°C): "))
    beta_12 = float(input("Enter moisture expansion coefficient in beta_12 (%/°C): "))
    return E1, E2, G12, nu12, alpha_1, alpha_2, alpha_12, beta_1, beta_2, beta_12

def calculate_Q(E1, E2, G12, nu12):
    # Calculate transformed stiffness matrix Q
    nu21 = (E2 / E1) * nu12
    Q11 = 10**9*E1 / (1 - nu12 * nu21)
    Q12 = nu21 * Q11
    Q22 = 10**9*E2 / (1 - nu12 * nu21)
    Q66 = 10**9*G12
    Q = np.array([[Q11, Q12, 0], [Q12, Q22, 0], [0, 0, Q66]])
    return Q

def calculate_Q_bar(Q, theta):
    # Calculate reduced stiffness matrix Q_bar
    theta = math.radians(theta)
    c = math.cos(theta)
    s = math.sin(theta)
    Q_bar = np.zeros((3, 3))
    Q_bar[0, 0] = Q[0, 0] * c**4 + (2 * Q[0, 1] + 4 * Q[2, 2]) * c**2 * s**2 + Q[1, 1] * s**4
    Q_bar[0, 1] = (Q[0, 0] + Q[1, 1] - 4 * Q[2, 2]) * c**2 * s**2 + Q[0, 1] * (c**4 + s**4)
    Q_bar[1, 1] = Q[0, 0] * s**4 + (2 * Q[0, 1] + 4 * Q[2, 2]) * c**2 * s**2 + Q[1, 1] * c**4
    Q_bar[0, 2] = (Q[0, 0] - Q[0, 1] - 2 * Q[2, 2]) * c**3 * s + (Q[1, 1] - Q[0, 1] - 2 * Q[2, 2]) * c * s**3
    Q_bar[1, 2] = (Q[0, 0] - Q[0, 1] - 2 * Q[2, 2]) * c * s**3 + (Q[1, 1] - Q[0, 1] - 2 * Q[2, 2]) * c**3 * s
    Q_bar[2, 2] = (Q[0, 0] + Q[1, 1] - 2 * Q[0, 1] - 2 * Q[2, 2]) * c**2 * s**2 + Q[2, 2] * (c**4 + s**4)
    Q_bar[1, 0] = Q_bar[0, 1]
    Q_bar[2, 0] = Q_bar[0, 2]
    Q_bar[2, 1] = Q_bar[1, 2]
    return Q_bar

def calculate_ABD(Q, zk_1, zk):
    # Calculate A, B, D matrices
    A = Q * (zk - zk_1)
    B = (1 / 2) * Q * ((zk ** 2) - (zk_1 ** 2))
    D = (1 / 3) * Q * ((zk ** 3) - (zk_1 ** 3))
    return A, B, D

def invert_ABBD(A, B, D):
    # Invert the ABBD matrix
    ABBD = np.zeros((6, 6))
    ABBD[:3, :3] = A
    ABBD[:3, 3:] = B
    ABBD[3:, :3] = B
    ABBD[3:, 3:] = D

    ABBD_inv = np.linalg.inv(ABBD)
    return ABBD_inv

def transformation_matrix(theta):
    theta = math.radians(theta)
    c = math.cos(theta)
    s = math.sin(theta)
    T = np.array([[c**2, s**2, 2*c*s],
                  [s**2, c**2, -2*c*s],
                  [-c*s, c*s, c**2 - s**2]]) 
    return T
    

def get_laminate(E1, E2, G12, nu12, alpha, beta, delta_T, delta_C, sequence):
    n = math.ceil(len(sequence)/2)
    i = 0
    e = 0
    A = np.zeros([3, 3], dtype=float)
    B = np.zeros([3, 3], dtype=float)
    D = np.zeros([3, 3], dtype=float)
    z = []
    Nt_eq = np.zeros(3, dtype=float)
    Mt_eq = np.zeros(3, dtype=float)
    Nc_eq = np.zeros(3, dtype=float)
    Mc_eq = np.zeros(3, dtype=float)
    Q = calculate_Q(E1, E2, G12, nu12)
    while i < len(sequence):
        if n > 0:
            theta = sequence[i]
            zk_1 = -n * t
            zk = -(n-1) * t
            z.append(zk_1)
            Q_bar = calculate_Q_bar(Q, theta)
            alpha = np.matmul(np.linalg.inv(transformation_matrix(theta)), alpha) 
            beta = np.matmul(np.linalg.inv(transformation_matrix(theta)), beta)  
            Nt_eq += delta_T * (zk - zk_1) * np.matmul(Q_bar, alpha)
            Nc_eq += delta_C * (zk - zk_1) * np.matmul(Q_bar, beta)
            Mt_eq += 0.5 * delta_T * ((zk ** 2) - (zk_1 ** 2)) * np.matmul(Q_bar, alpha)
            Mc_eq += 0.5 * delta_C * ((zk ** 2) - (zk_1 ** 2)) * np.matmul(Q_bar, beta)
            a, b, d = calculate_ABD(Q_bar, zk_1, zk)
            n -= 1
        else:
            theta = sequence[i]
            zk_1 = e * t
            zk = (e+1) * t
            z.append(zk)
            Q_bar = calculate_Q_bar(Q, theta)
            alpha = np.matmul(np.linalg.inv(transformation_matrix(theta)), alpha) 
            beta = np.matmul(np.linalg.inv(transformation_matrix(theta)), beta) 
            Nt_eq += delta_T * (zk - zk_1) * np.matmul(Q_bar, alpha)
            Nc_eq += delta_C * (zk - zk_1) * np.matmul(Q_bar, beta)
            Mt_eq += 0.5 * delta_T * ((zk ** 2) - (zk_1 ** 2)) * np.matmul(Q_bar, alpha)
            Mc_eq += 0.5 * delta_C * ((zk ** 2) - (zk_1 ** 2)) * np.matmul(Q_bar, beta)
            a, b, d = calculate_ABD(Q_bar, zk_1, zk)
            e += 1
        A += a
        B += b
        D += d
        i += 1
    laminate = {'A': A, 'B': B, 'D': D, 'Nt_eq': Nt_eq, 'Nc_eq':  Nc_eq, 'Mt_eq': Mt_eq, 'Mc_eq': Mc_eq, 'z': z}
    return laminate


def calculate_mid_surface_strains_curvatures(laminate, load_vec, delta_T, delta_C):
    # Calculate mid-surface strains and curvatures
    A = laminate['A']  # Lamina A matrix
    B = laminate['B']  # Lamina B matrix
    D = laminate['D']  # Lamina D matrix
    thermal_load = np.concatenate((laminate['Nt_eq'], laminate['Mt_eq']), axis=0)
    hygro_load = np.concatenate((laminate['Nc_eq'], laminate['Mc_eq']), axis=0)
    
    load = load_vec + thermal_load + hygro_load
    strains_curvatures = np.matmul(invert_ABBD(A, B, D), load.T) 

    return strains_curvatures

def calculate_strains(strains_curvatures, laminate):
    mid_surface_strain = strains_curvatures[:3]
    curvature = strains_curvatures[3:]
    z = laminate['z']
    n = len(z)
    strains = np.empty((n, 3))  
    for i in range(n):
        strain = mid_surface_strain + z[i] * curvature
        strain[2] /= 2 
        strains[i] = strain
        
    return strains 


def strain_material_axis(strains, sequence):
    r,c = strains.shape
    strains_ma = np.zeros((r,c))
    for i in range(r):
        T = transformation_matrix(sequence[i])
        strains_ma[i] = np.matmul(T, strains[i])
    return strains_ma

def calculate_stresses(Q, strains):
    r,c  = strains.shape
    stresses = np.zeros((r,c))
    for i in range(r):
        strains[i][2] *= 2
        stresses[i] = np.matmul(Q, strains[i])
    return stresses


def stress_ratios(stresses,sigma_1_tensile, sigma_1_compressive, sigma_2_tensile, sigma_2_compressive, tau_12):
    stress_ratios = np.zeros([len(stresses),3])
    row = 0
    for stress in stresses:
        stress_ratios[row][0] = stress[0]/sigma_1_tensile if stress[0]> 0 else stress[0]/sigma_1_compressive
        stress_ratios[row][1] = stress[1]/sigma_2_tensile if stress[1]> 0 else stress[1]/sigma_2_compressive
        stress_ratios[row][2] = stress[2]/tau_12
        row += 1
            

    return np.max(stress_ratios), row


def get_applied_load():
    # Get applied loads from user
    Nx = float(input("Enter applied load in x-direction (N/mm): "))
    Ny = float(input("Enter applied load in y-direction (N/mm): "))
    Nxy = float(input("Enter applied shear load (N/mm): "))
    Mx = float(input("Enter applied moment about x-axis (N): "))
    My = float(input("Enter applied moment about y-axis (N): "))
    Mxy = float(input("Enter applied twisting moment (N): "))
    return Nx, Ny, Nxy, Mx, My, Mxy

def get_environmental_changes():
    # Get environmental changes from user
    delta_T = float(input("Enter temperature change (°C): "))
    delta_C = float(input("Enter moisture change (%): "))
    return delta_T, delta_C

def get_strength_properties():
    # Get strength properties from user
    sigma_1_tensile = float(input("Enter tensile strength (sigma_1) (MPa): "))
    sigma_1_compressive = float(input("Enter compressive strength (sigma_1) (MPa): "))
    sigma_2_tensile = float(input("Enter tensile strength (sigma_2) (MPa): "))
    sigma_2_compressive = float(input("Enter compressive strength (sigma_2) (MPa): "))
    tau_12 = float(input("Enter shear strength (tau_12) (MPa): "))
    return sigma_1_tensile, sigma_1_compressive, sigma_2_tensile, sigma_2_compressive, tau_12



# Get user input
#E1, E2, G12, nu12, alpha_1, alpha_2, alpha_12, beta_1, beta_2, beta_12= get_material_properties()
E1, E2, G12, nu12, alpha_1, alpha_2, alpha_12, beta_1, beta_2, beta_12 = 140, 10, 7, 0.3, 0, 0, 0, 0, 0, 0
#Nx, Ny, Nxy, Mx, My, Mxy = get_applied_load()
Nx, Ny, Nxy, Mx, My, Mxy  = 100000, 0, 0, 0, 0, 0
load_vec = np.array([Nx, Ny, Nxy, Mx, My, Mxy])
#input_sequence = input("Enter stacking sequence of the laminate (e.g., 0 45 -45 90): ").split()
input_sequence = '0 90 90 0'.split()
sequence = [float(v) for v in input_sequence]
#t = float(input("Enter thickness of the lamina (mm): "))
t = 0.25*10**(-3)
#delta_T, delta_C = get_environmental_changes()
delta_T, delta_C = 0, 0
#sigma_1_tensile, sigma_1_compressive, sigma_2_tensile, sigma_2_compressive, tau_12 = get_strength_properties()
sigma_1_tensile, sigma_1_compressive, sigma_2_tensile, sigma_2_compressive, tau_12 = 1400*10**6, 1400*10**6, 50*10**6, 200*10**6, 70*10**6
alpha  = np.array([alpha_1, alpha_2, alpha_12/2])
beta = np.array([beta_1, beta_2, beta_12/2])

laminate_props = get_laminate(E1, E2, G12, nu12, alpha, beta, delta_T, delta_C, sequence)


# Calculate mid-surface strains and curvatures
strains_curvatures = calculate_mid_surface_strains_curvatures(laminate_props, load_vec, delta_T, delta_C)

# Calculate strains
strains = calculate_strains(strains_curvatures, laminate_props)


strains_ma = strain_material_axis(strains, sequence)

Q = calculate_Q(E1, E2, G12, nu12)

# Calculate stresses for the last lamina
stresses = calculate_stresses(Q, strains_ma)
# Calculate FPF and LPF
max_stress_ratio, layer = stress_ratios(stresses, sigma_1_tensile, sigma_1_compressive, sigma_2_tensile, sigma_2_compressive, tau_12)

FPL = 100/max_stress_ratio
print("First ply load (FPF) in kN:", FPL)




First ply load (FPF) in kN: 378.9473684210526
